# Soulection Spotify Playlists
This Jupyter Notebook has been created to code and test a tool that automatically creates a Spotify
 playlist from a Soulection Radio show using the Soulection Radio show ID.

In [1]:
# Import modules
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import spotipy
import spotipy.util as util

In [2]:
# Function to return ids of Spotify songs
USER_AGENT = 'User-Agent'
MOZILLA_GENERAL_TOKEN = 'Mozilla/5.0'
HTML_SPOTIFY_TITLE = 'spotify'
HREF_ATTRIBUTE = 'href'
SPLITTER = 'track/'

URL = 'https://soulectiontracklists.com/episodes/'
HEADER = {USER_AGENT:MOZILLA_GENERAL_TOKEN}

SPOTIFY_HREFS_LIST = list()
SPOTIFY_IDS_LIST = list()

def loop_spotify_titles(spotify_id_list, spotify_title_soup):
    """

    :param spotify_id_list:
    :param spotify_title_soup:
    :return:
    """
    for spotify_title in spotify_title_soup:
        spotify_id_list.append(spotify_title[HREF_ATTRIBUTE])

    return spotify_id_list

def loop_spotify_ids(spotify_ids_list, spotify_href_list):
    """

    :param spotify_ids_list:
    :param spotify_href_list:
    :return:
    """
    for spotify_song_id in spotify_href_list:
        spotify_ids_list.append(spotify_song_id.split(SPLITTER)[1])

    return spotify_ids_list


def soulection_show_spotify_song_ids(SHOW_ID):
    """

    :param SHOW_ID:
    :return:
    """
    soulection_tracklist_url = URL + str(SHOW_ID)
    request = Request(soulection_tracklist_url, headers=HEADER)
    soulection_tracklist_url_response = urlopen(request)
    soup = BeautifulSoup(soulection_tracklist_url_response)
    spotify_title_soup = soup.find_all(title=HTML_SPOTIFY_TITLE)

    spotify_href_list = loop_spotify_titles(SPOTIFY_HREFS_LIST, spotify_title_soup)
    spotify_id_list = loop_spotify_ids(SPOTIFY_IDS_LIST, spotify_href_list)

    return spotify_id_list

In [3]:
# Return ids of Spotify songs on Soulection show 481
show_481_spotify_ids = soulection_show_spotify_song_ids(481)

In [29]:
# Function for Spotipy authentication
SPOTIFY_API_CREDENTIALS = pd.read_csv('spotify_api_keys.csv', header=None)

SPOTIFY_USER_ID = SPOTIFY_API_CREDENTIALS[1].iloc[0]
SPOTIFY_CLIENT_ID = SPOTIFY_API_CREDENTIALS[1].iloc[1]
SPOTIFY_CLIENT_SECRET = SPOTIFY_API_CREDENTIALS[1].iloc[2]
REDIRECT_URL = 'http://127.0.0.1:9090'

## All scope
SPOTIFY_AUTHORIZATION_SCOPE = 'ugc-image-upload user-read-playback-state streaming ' \
                              'user-read-email playlist-read-collaborative '  \
                              'user-modify-playback-state user-read-private  ' \
                              'playlist-modify-public user-library-modify user-top-read ' \
                              'user-read-playback-position user-read-currently-playing ' \
                              'playlist-read-private user-follow-read app-remote-control ' \
                              'user-read-recently-played user-follow-modify user-library-read'

def spotify_authentication(spotify_user_id, spotify_authorization_scope, spotify_client_id,
                           spotify_client_secret, redicrect_url):
    """

    :param spotify_user_id:
    :param spotify_authorization_scope:
    :param spotify_client_id:
    :param spotify_client_secret:
    :param redicrect_url:
    :return:
    """
    token = util.prompt_for_user_token(
        spotify_user_id,
        spotify_authorization_scope,
        client_id = spotify_client_id,
        client_secret = spotify_client_secret,
        redirect_uri=redicrect_url)

    sp = spotipy.Spotify(auth=token)

    return sp

In [30]:
# Spotipy authentication
sp = spotify_authentication(SPOTIFY_USER_ID, SPOTIFY_AUTHORIZATION_SCOPE, SPOTIFY_CLIENT_ID,
                       SPOTIFY_CLIENT_SECRET, REDIRECT_URL)


In [6]:
SOULECTION_SHOW_SPOTIFY_IDS = show_481_spotify_ids
PLAYLIST_PREFFIX = 'Soulection Show'
SOULECTION_SHOW_NUMBER = 481
PLAYLIST_NAME = PLAYLIST_PREFFIX + ' ' + str(SOULECTION_SHOW_NUMBER)

def create_and_add_tracks_to_playlist(spotify_song_ids, playlist_name):
    """

    :param spotify_song_ids:
    :param playlist_name:
    :return:
    """
    playlist = sp.user_playlist_create(SPOTIFY_USER_ID, playlist_name)
    sp.playlist_add_items(playlist['id'], spotify_song_ids)

    print(f'Spotify playlist "{playlist_name}" successfully created!')
    pass

In [7]:
create_and_add_tracks_to_playlist(SOULECTION_SHOW_SPOTIFY_IDS, PLAYLIST_NAME)

Spotify playlist "Soulection Show 481" successfully created!


In [ ]:
# Deletes playlist 'Pythonically Created' using Playlist Id
sp.current_user_unfollow_playlist('3Ti4AOTc9SiJgRRpWJ5gVV')

## Download Show Art and Upload as Spotify Cover Art

In [ ]:
# Find show art
soulection_tracklist_url = URL + str(481)
request = Request(soulection_tracklist_url, headers=HEADER)
soulection_tracklist_url_response = urlopen(request)
soup = BeautifulSoup(soulection_tracklist_url_response)
soup.find_all(alt="Show #481")


In [43]:
import requests
f = open(rf'C:\Users\Jaume\Documents\Python Projects\soulection_playlists\{PLAYLIST_NAME}.jpeg',
         'wb')
f.write(requests.get('https://firebase.soulectiontracklists'
                      '.com/cdn/image//t_square_medium/images/episodes/481.jpg').content)
f.close()

In [44]:
# Function to turn image into Base64
import base64

def get_base64_encoded_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

In [45]:
# Get Base64 string
base64_string = get_base64_encoded_image(
    rf'C:\Users\Jaume\Documents\Python Projects\soulection_playlists\{PLAYLIST_NAME}.jpeg')

In [46]:
sp.playlist_upload_cover_image('4vVdLnQ3xcPMRpQhq9614t', base64_string)

In [32]:
base64_string


'UklGRgQwAABXRUJQVlA4IPgvAAAwrgCdASoAAQABPpE+mEkloyIhKrZskLASCWVmF8x6hSge9Lq6mPItT50ysvLMfB56n8JceEj51/F79X4O+eX6RIreiP2nUm8H/5Pra/xe/P5rain5V/UfO7g5dify/QR8D+cx3Q9Lf4T1AvzD9jPBPoA/1T/E/+L/Ne1F/8edT9r/5vsGfsR10fRuaG/hDxXMgzok9UIfXfsDEoLvNdceEzOs+AEDzIT0dXY6J5JeonZyy3lUarv1trmh16fP0/zhfmK7yjKXvI/RJ1i/3KAWhbRuSCtdAXONoYPeUVQ2UhK3Bb6l3r+NYUCRloTPiT4MVioHqZqRwKek7iX1KA+cFS+GCwypKi1LBhDa9zgg890uod9hf2QKmAIVfGhWxn6vnMVVgBU/pWlr90NLQZOHmZWPTvAzLyaXdm6TlXqr/WdwIB6302OhyO4AgbRuGNmrebm2wXFBrZy50rSonNiitKcbFmsAzP6rRCS4VGEnUDxrDITa/WUmnNnM5/2pTD4IDoCS5KSl7DiKSCNqnoUC6AjRPvSuU17IsJL8SP6yOTRmv0kWFt4b8RLc/2kFuAyXkd4k2fllHCF2J5sFRftb694Gy4Qb55k2jLsmN+6d7uMLWjUx6gimahyjsBxf8GGE9W9EUptWoosiOJQ06p5AQa18MkL6Ib/UIY0qfBSNXQmK4ysz0pg7ysmdlqus7jCG0t7TZI9hmL3RYYxSD+C3PG2ZznkOCYJOtRBnyIwzVId0iWcuWqoEPWFDe0FRepExfFxF8CDUlfzYzARo4eN+071bOB2hD/6XHyJgsNSrDQcrmCgJ3pQOKYBMEHiPUOW3kdU/qzVaoKbhjjDEr92nvuSZ/vgqOOln+s1PSKjesxI7HqtcqfhMm4P1/75hreEgvPT84NzmqHQeB8ffQ/IJkzZdAwtcGcNJfvCvn6CJu2PsYgJAYHTd6VFDeAwlGc8uofioTGKRsuM